In [1]:
#Import libraries
import webbrowser
import os
import glob
import chardet
import datetime
import numpy as np
import pandas as pd

In [2]:
stations={
    'Paya Lebar': {
        'code':      '06',
        'latitude':  1.3524, 
        'longitude': 103.9007},
    'Tengah':{
        'code':      '23',
        'latitude':  1.3858,
        'longitude': 103.7114},
    'Changi':{
        'code':      '24',
        'latitude':  1.3678, 
        'longitude': 103.9826},
    'Seletar':{
        'code':      '25',
        'latitude':  1.4166, 
        'longitude': 103.8654},
    'Tai Seng':{
        'code':      '43',
        'latitude':  1.3399, 
        'longitude': 103.8878},
    'Jurong (West)':{
        'code':      '44',
        'latitude':  1.3455 , 
        'longitude': 103.6806 },
    'Ang Mo Kio':{
        'code':      '109',
        'latitude':  1.3764  , 
        'longitude': 103.8492},
    'Clementi':{
        'code':      '50',
        'latitude':  1.3337, 
        'longitude': 103.7768},
    'Admiralty':{
        'code':      '104',
        'latitude':  1.4439, 
        'longitude': 103.7854},
    'Sentosa Island':{
        'code':      '60',
        'latitude':  1.2500, 
        'longitude': 103.8279},
    'Sembawang':{
        'code':      '80',
        'latitude':  1.4252, 
        'longitude': 103.8202},
    'Boon Lay (East)':{
        'code':      '86',
        'latitude':  1.3302, 
        'longitude': 103.7205},
    'Semakau Island':{
        'code':      '102',
        'latitude':  1.1890, 
        'longitude': 103.7680},
    'Pulau Ubin':{
        'code':      '106',
        'latitude':  1.4168, 
        'longitude': 103.9673},
    'East Coast Parkway':{
        'code':      '107',
        'latitude':  1.3134, 
        'longitude': 103.9619},
    'Marina Barrage':{
        'code':      '108',
        'latitude':  1.2799, 
        'longitude': 103.8703},
    'Newton':{
        'code':      '111',
        'latitude':  1.3106, 
        'longitude': 103.8365},
    'Tuas South':{
        'code':      '115',
        'latitude':  1.2938, 
        'longitude': 103.6184},
    'Pasir Panjang':{
        'code':      '116',
        'latitude':  1.2810, 
        'longitude': 103.7540},
    'Jurong Island':{
        'code':      '117',
        'latitude':  1.2542, 
        'longitude': 103.6741},
    'Choa Chu Kang (South)':{
        'code':      '121',
        'latitude':  1.3729, 
        'longitude': 103.722},
    'Khatib':{
        'code':      '122',
        'latitude':  1.4173, 
        'longitude': 103.8249}
}

Missing files:
S121_201505
S111_201611
S111_201610
S111_201609
S106_201311
S106_201312
S106_201401
S106_201402

S102_201312
S102_201411
S102_201401
S86_202012
...
S86_201602
S60_201612

S104_201409
S44_201412
S44_201411
S44_201410
S44_201409
S44_201408
S44_201407


In [3]:
weather_df=pd.DataFrame()
# Append all 2,035 monthly files together
for station in stations.keys():
    df_append = pd.DataFrame()
    code=stations[station]['code']
    path = "../assets/climate_historical_daily/DAILYDATA_S"+code+"_*.csv"

    for file in glob.glob(path):
        with open(file,'rb') as f:
            enc = chardet.detect(f.read())
        df_temp = pd.read_csv(file,encoding = enc['encoding']) # otherwise will have uft-8 error as is in ISO-8859-1 format
        df_temp = df_temp.rename(columns=str.lower) # columns names in different case between files
        df_append = pd.concat(objs =[df_append,df_temp],axis=0)
        
    df_append['station_check']=station
    df_append['station_code']=code
    df_append['latitude']=stations[station]['latitude']
    df_append['longitude']=stations[station]['longitude']
    weather_df = pd.concat(objs =[weather_df,df_append],axis=0)

In [4]:
# double check if station code and station name is aligned.
weather_df['check']=weather_df['station']==weather_df['station_check']
weather_df['check'].value_counts()

True    61939
Name: check, dtype: int64

In [5]:
weather_df=weather_df.drop(['station_check','check'],axis=1)

In [6]:
new_columns_dict = {
    'station': 'station',
    'year':'year',
    'month': 'month',
    'day': 'day',
    'daily rainfall total (mm)': 'total_daily_rainfall',
    'highest 30 min rainfall (mm)': 'highest_30min_rainfall',
    'highest 60 min rainfall (mm)': 'highest_60min_rainfall',
    'highest 120 min rainfall (mm)': 'highest_120min_rainfall',
    'mean temperature (°c)': 'mean_temp',
    'maximum temperature (°c)': 'max_temp',
    'minimum temperature (°c)': 'min_temp',
    'mean wind speed (km/h)': 'mean_wind_sp',
    'max wind speed (km/h)': 'max_wind_sp'
}

weather_df.rename(columns=new_columns_dict, inplace=True)

In [7]:
weather_df.isna().sum()

station                      0
year                        96
month                       96
day                         96
total_daily_rainfall         0
highest_30min_rainfall       0
highest_60min_rainfall       0
highest_120min_rainfall      0
mean_temp                    0
max_temp                   235
min_temp                   247
mean_wind_sp                 0
max_wind_sp                274
station_code                 0
latitude                     0
longitude                    0
dtype: int64

In [8]:
weather_df=weather_df.dropna(subset=['year', 'month', 'day'], how='any', axis=0)

In [9]:
weather_df['date'] = pd.to_datetime(weather_df[['year', 'month', 'day']])

In [10]:
weather_df['week'] = weather_df['date'].apply(lambda x: x.isocalendar()[1])

In [11]:
# align with cluster.csv time period
start_date = '2013-05-23'
end_date = '2020-11-06'

# filter the dataframe to keep only observations within the date range
weather_df = weather_df[(weather_df['date'] >= start_date) & (weather_df['date'] <= end_date)].copy(deep=True)

In [12]:
weather_df['date'].describe(datetime_is_numeric=True)

count                            57564
mean     2017-02-04 14:22:08.330206464
min                2013-05-23 00:00:00
25%                2015-03-27 00:00:00
50%                2017-02-02 00:00:00
75%                2018-12-19 00:00:00
max                2020-11-06 00:00:00
Name: date, dtype: object

In [13]:
col_list=['total_daily_rainfall','highest_30min_rainfall','highest_60min_rainfall','highest_120min_rainfall',
          'mean_temp','max_temp','min_temp',
          'mean_wind_sp','max_wind_sp']
for col in col_list:
    weather_df[col] = weather_df[col].map(lambda x: str(x).strip()).replace(['—','-'], np.nan).map(lambda x: float(x)) #different '-'s

In [14]:
weather_df['rainy_day'] = weather_df['total_daily_rainfall'].apply(lambda x: 1 if x>=0.2 else 0)

In [15]:
weather_df.head()

,station,year,month,day,total_daily_rainfall,highest_30min_rainfall,highest_60min_rainfall,highest_120min_rainfall,mean_temp,max_temp,min_temp,mean_wind_sp,max_wind_sp,station_code,latitude,longitude,date,week,rainy_day
0,Paya Lebar,2018.0,1.0,1.0,33.2,NaN,NaN,NaN,25.2,27.3,23.8,7.2,27.7,06,1.3524,103.9007,2018-01-01,1,1
1,Paya Lebar,2018.0,1.0,2.0,10.2,NaN,NaN,NaN,25.8,28.4,24.1,8.6,25.9,06,1.3524,103.9007,2018-01-02,1,1
2,Paya Lebar,2018.0,1.0,3.0,62.2,NaN,NaN,NaN,26.8,32.1,24.5,7.6,31.3,06,1.3524,103.9007,2018-01-03,1,1
3,Paya Lebar,2018.0,1.0,4.0,0.3,NaN,NaN,NaN,26.9,29.5,25.0,9.4,33.5,06,1.3524,103.9007,2018-01-04,1,1
4,Paya Lebar,2018.0,1.0,5.0,1.5,NaN,NaN,NaN,26.8,30.0,24.3,9.0,24.1,06,1.3524,103.9007,2018-01-05,1,1


In [16]:
# create a function to find region of a location using its coordinates:

def get_region(lat, lng):
    
    # define the boundaries of each region in Singapore

    central_region = [(1.2848, 103.7722), (1.3199, 103.9070)]
    east_region = [(1.3200, 103.8862), (1.3730, 103.9910)]
    north_region = [(1.4153, 103.7019), (1.4543, 103.8205)]
    northeast_region = [(1.3600, 103.8536), (1.3984, 103.9200)]
    west_region = [(1.2058, 103.6953), (1.3912, 103.8353)]


    # check if the given coordinates fall within one of the boundaries
    if lat <= 1.3200 and  103.7722 <= lng <= 103.9070:
        return 'Central'
    elif lat <= 1.3730 and 103.8862 <= lng:
        return 'East'
    elif 1.32 <= lat and lng <= 104:
        return 'North'
    elif lng <= 103.7722:
        return 'West'
    elif 1.356 <= lat and 103.8536 <= lng:
        return 'Northeast'
    else:
        return 'Unknown'

In [17]:
# add a new column 'region' with the corresponding region for each coordinate
weather_df['region'] = weather_df.apply(lambda row: get_region(row['latitude'],
                                                               row['longitude']), 
                                        axis=1)

In [18]:
weather_df['region'].value_counts()

North      27869
East       10900
West       10742
Central     8053
Name: region, dtype: int64

In [19]:
counts = weather_df.groupby(['region', 'station']).size()
counts

region   station              
Central  Marina Barrage           2725
         Newton                   2634
         Sentosa Island           2694
East     Changi                   2725
         East Coast Parkway       2725
         Paya Lebar               2725
         Tai Seng                 2725
North    Admiralty                2695
         Ang Mo Kio               2725
         Boon Lay (East)           984
         Choa Chu Kang (South)    2694
         Clementi                 2725
         Jurong (West)            2541
         Khatib                   2725
         Pulau Ubin               2605
         Seletar                  2725
         Sembawang                2725
         Tengah                   2725
West     Jurong Island            2725
         Pasir Panjang            2725
         Semakau Island           2567
         Tuas South               2725
dtype: int64

In [21]:
weather_df.to_csv('../assets/climate_historical_daily_2013-2019.csv',index=False)

In [22]:
weather_df=weather_df.drop(['month','day','station_code', 'latitude', 'longitude', 'date'],axis=1)

In [23]:
aggweather_df = weather_df.groupby(['year', 'week', 'region']).agg({'total_daily_rainfall': 'sum',
                                                                    'highest_30min_rainfall': 'max',
                                                                    'highest_60min_rainfall': 'max',
                                                                    'highest_120min_rainfall': 'max',
                                                                    'max_wind_sp': 'max',
                                                                    'max_temp': 'max',
                                                                    'rainy_day': 'max',
                                                                    'mean_temp': 'mean',
                                                                    'mean_wind_sp': 'mean',
                                                                    'min_temp': 'min'})

In [24]:
aggweather_df=aggweather_df.reset_index()
aggweather_df

,year,week,region,total_daily_rainfall,highest_30min_rainfall,highest_60min_rainfall,highest_120min_rainfall,max_wind_sp,max_temp,rainy_day,mean_temp,mean_wind_sp,min_temp
0,2013.0,1,Central,2.6,NaN,NaN,NaN,48.6,30.6,1,26.675000,8.920000,24.8
1,2013.0,1,East,28.0,NaN,NaN,NaN,39.2,30.4,1,26.483333,12.157143,24.8
2,2013.0,1,North,50.2,NaN,NaN,NaN,40.7,31.5,1,26.000000,9.289474,23.6
3,2013.0,1,West,22.8,NaN,NaN,NaN,53.3,30.7,1,26.550000,9.250000,24.6
4,2013.0,21,Central,72.4,NaN,NaN,NaN,43.9,33.6,1,28.466667,4.916667,23.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1563,2020.0,44,West,162.8,41.6,51.2,71.2,65.9,32.8,1,28.433333,10.290476,23.9
1564,2020.0,45,Central,110.0,18.8,19.2,19.8,49.3,33.9,1,27.600000,4.826667,22.5
1565,2020.0,45,East,549.5,61.8,102.6,111.6,53.7,34.2,1,27.665000,6.925000,22.6
1566,2020.0,45,North,399.0,31.2,41.2,45.4,52.6,34.9,1,27.125714,5.765714,22.9


In [25]:
aggweather_df.to_csv('../assets/agg_climate_2013-2019.csv',index=False)